In [14]:
import pandas as pd
import pickle
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np


from surprise import Dataset, Reader, accuracy, NormalPredictor, KNNBasic, KNNWithMeans, KNNWithZScore, KNNBaseline, SVD, BaselineOnly, SVDpp, NMF, SlopeOne, CoClustering
from surprise.accuracy import rmse
from surprise.prediction_algorithms import SVD, SVDpp, NMF, BaselineOnly, NormalPredictor
from IPython.core.display import HTML
from surprise.model_selection.split import train_test_split as surprise_train_test_split
from surprise.model_selection import GridSearchCV, cross_validate

from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer
from nltk import FreqDist
from nltk import word_tokenize
from nltk.corpus import stopwords
import string
from nltk.collocations import *
import re
import nltk

%matplotlib inline

In [15]:
recipes_df = pd.read_csv("./Data/RAW_recipes.csv")

In [123]:
recipes_df.head(10)

,name,id,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients
0,arriba baked winter squash mexican style,137739,55,47892,2005-09-16,"['60-minutes-or-less', 'time-to-make', 'course...","[51.5, 0.0, 13.0, 0.0, 2.0, 0.0, 4.0]",11,"['make a choice and proceed with recipe', 'dep...",autumn is my favorite time of year to cook! th...,"['winter squash', 'mexican seasoning', 'mixed ...",7
1,a bit different breakfast pizza,31490,30,26278,2002-06-17,"['30-minutes-or-less', 'time-to-make', 'course...","[173.4, 18.0, 0.0, 17.0, 22.0, 35.0, 1.0]",9,"['preheat oven to 425 degrees f', 'press dough...",this recipe calls for the crust to be prebaked...,"['prepared pizza crust', 'sausage patty', 'egg...",6
2,all in the kitchen chili,112140,130,196586,2005-02-25,"['time-to-make', 'course', 'preparation', 'mai...","[269.8, 22.0, 32.0, 48.0, 39.0, 27.0, 5.0]",6,"['brown ground beef in large pot', 'add choppe...",this modified version of 'mom's' chili was a h...,"['ground beef', 'yellow onions', 'diced tomato...",13
3,alouette potatoes,59389,45,68585,2003-04-14,"['60-minutes-or-less', 'time-to-make', 'course...","[368.1, 17.0, 10.0, 2.0, 14.0, 8.0, 20.0]",11,['place potatoes in a large pot of lightly sal...,"this is a super easy, great tasting, make ahea...","['spreadable cheese with garlic and herbs', 'n...",11
4,amish tomato ketchup for canning,44061,190,41706,2002-10-25,"['weeknight', 'time-to-make', 'course', 'main-...","[352.9, 1.0, 337.0, 23.0, 3.0, 0.0, 28.0]",5,['mix all ingredients& boil for 2 1 / 2 hours ...,my dh's amish mother raised him on this recipe...,"['tomato juice', 'apple cider vinegar', 'sugar...",8
5,apple a day milk shake,5289,0,1533,1999-12-06,"['15-minutes-or-less', 'time-to-make', 'course...","[160.2, 10.0, 55.0, 3.0, 9.0, 20.0, 7.0]",4,"['combine ingredients in blender', 'cover and ...",NaN,"['milk', 'vanilla ice cream', 'frozen apple ju...",4
6,aww marinated olives,25274,15,21730,2002-04-14,"['15-minutes-or-less', 'time-to-make', 'course...","[380.7, 53.0, 7.0, 24.0, 6.0, 24.0, 6.0]",4,['toast the fennel seeds and lightly crush the...,my italian mil was thoroughly impressed by my ...,"['fennel seeds', 'green olives', 'ripe olives'...",9
7,backyard style barbecued ribs,67888,120,10404,2003-07-30,"['weeknight', 'time-to-make', 'course', 'main-...","[1109.5, 83.0, 378.0, 275.0, 96.0, 86.0, 36.0]",10,['in a medium saucepan combine all the ingredi...,this recipe is posted by request and was origi...,"['pork spareribs', 'soy sauce', 'fresh garlic'...",22
8,bananas 4 ice cream pie,70971,180,102353,2003-09-10,"['weeknight', 'time-to-make', 'course', 'main-...","[4270.8, 254.0, 1306.0, 111.0, 127.0, 431.0, 2...",8,"['crumble cookies into a 9-inch pie plate , or...",NaN,"['chocolate sandwich style cookies', 'chocolat...",6
9,beat this banana bread,75452,70,15892,2003-11-04,"['weeknight', 'time-to-make', 'course', 'main-...","[2669.3, 160.0, 976.0, 107.0, 62.0, 310.0, 138.0]",12,"['preheat oven to 350 degrees', 'butter two 9x...",from ann hodgman's,"['sugar', 'unsalted butter', 'bananas', 'eggs'...",9


In [63]:
recipes_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 231637 entries, 0 to 231636
Data columns (total 12 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   name            231636 non-null  object
 1   id              231637 non-null  int64 
 2   minutes         231637 non-null  int64 
 3   contributor_id  231637 non-null  int64 
 4   submitted       231637 non-null  object
 5   tags            231637 non-null  object
 6   nutrition       231637 non-null  object
 7   n_steps         231637 non-null  int64 
 8   steps           231637 non-null  object
 9   description     226658 non-null  object
 10  ingredients     231637 non-null  object
 11  n_ingredients   231637 non-null  int64 
dtypes: int64(5), object(7)
memory usage: 21.2+ MB


In [16]:
# Change rename id to recipe_id and change to string
recipes_df.rename(columns= {"id": "recipe_id"}, inplace=True)
recipes_df["recipe_id"] = recipes_df["recipe_id"].astype(str)
recipes_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 231637 entries, 0 to 231636
Data columns (total 12 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   name            231636 non-null  object
 1   recipe_id       231637 non-null  object
 2   minutes         231637 non-null  int64 
 3   contributor_id  231637 non-null  int64 
 4   submitted       231637 non-null  object
 5   tags            231637 non-null  object
 6   nutrition       231637 non-null  object
 7   n_steps         231637 non-null  int64 
 8   steps           231637 non-null  object
 9   description     226658 non-null  object
 10  ingredients     231637 non-null  object
 11  n_ingredients   231637 non-null  int64 
dtypes: int64(4), object(8)
memory usage: 21.2+ MB


In [ ]:
# Things to do: 
# Create column on calories
# Nutrition -- create categories (healthy vs indulgent)
# 

## Feature Engineering - Nutrition Column

In [65]:
# The nutrition column contains a list of nutritional components
#[(calories (#), total fat (PDV), sugar (PDV) , sodium (PDV) , protein (PDV) , saturated fat (PDV), Carbohydrates(PDV)] 
# Pull out nutritional facts into own columns

print(recipes_df["nutrition"][0])

[51.5, 0.0, 13.0, 0.0, 2.0, 0.0, 4.0]


In [66]:
type(recipes_df["nutrition"][0])

str

In [17]:
#Create new columns that split on the nutrition column
recipes_df[["calories", "total_fat_pdv", "sugar_pdv", "sodium_pdv", "protein_pdv", "saturated_fat_pdv", "carbs_pdv"]]=recipes_df["nutrition"].str.split(",", expand=True)

In [16]:
recipes_df.head()

,name,recipe_id,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients,calories,total_fat_pdv,sugar_pdv,sodium_pdv,protein_pdv,saturated_fat_pdv,carbs_pdv
0,arriba baked winter squash mexican style,137739,55,47892,2005-09-16,"['60-minutes-or-less', 'time-to-make', 'course...","[51.5, 0.0, 13.0, 0.0, 2.0, 0.0, 4.0]",11,"['make a choice and proceed with recipe', 'dep...",autumn is my favorite time of year to cook! th...,"['winter squash', 'mexican seasoning', 'mixed ...",7,[51.5,0.0,13.0,0.0,2.0,0.0,4.0]
1,a bit different breakfast pizza,31490,30,26278,2002-06-17,"['30-minutes-or-less', 'time-to-make', 'course...","[173.4, 18.0, 0.0, 17.0, 22.0, 35.0, 1.0]",9,"['preheat oven to 425 degrees f', 'press dough...",this recipe calls for the crust to be prebaked...,"['prepared pizza crust', 'sausage patty', 'egg...",6,[173.4,18.0,0.0,17.0,22.0,35.0,1.0]
2,all in the kitchen chili,112140,130,196586,2005-02-25,"['time-to-make', 'course', 'preparation', 'mai...","[269.8, 22.0, 32.0, 48.0, 39.0, 27.0, 5.0]",6,"['brown ground beef in large pot', 'add choppe...",this modified version of 'mom's' chili was a h...,"['ground beef', 'yellow onions', 'diced tomato...",13,[269.8,22.0,32.0,48.0,39.0,27.0,5.0]
3,alouette potatoes,59389,45,68585,2003-04-14,"['60-minutes-or-less', 'time-to-make', 'course...","[368.1, 17.0, 10.0, 2.0, 14.0, 8.0, 20.0]",11,['place potatoes in a large pot of lightly sal...,"this is a super easy, great tasting, make ahea...","['spreadable cheese with garlic and herbs', 'n...",11,[368.1,17.0,10.0,2.0,14.0,8.0,20.0]
4,amish tomato ketchup for canning,44061,190,41706,2002-10-25,"['weeknight', 'time-to-make', 'course', 'main-...","[352.9, 1.0, 337.0, 23.0, 3.0, 0.0, 28.0]",5,['mix all ingredients& boil for 2 1 / 2 hours ...,my dh's amish mother raised him on this recipe...,"['tomato juice', 'apple cider vinegar', 'sugar...",8,[352.9,1.0,337.0,23.0,3.0,0.0,28.0]


In [18]:
#remove the open and closed brackets on calories and carbs respectively
#df['value'] = df['value'].str.strip('[]').astype(int)
recipes_df["calories"] = recipes_df["calories"].str.strip("[")
recipes_df["carbs_pdv"] = recipes_df["carbs_pdv"].str.strip("]")

#sanity check
recipes_df.head()

,name,recipe_id,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients,calories,total_fat_pdv,sugar_pdv,sodium_pdv,protein_pdv,saturated_fat_pdv,carbs_pdv
0,arriba baked winter squash mexican style,137739,55,47892,2005-09-16,"['60-minutes-or-less', 'time-to-make', 'course...","[51.5, 0.0, 13.0, 0.0, 2.0, 0.0, 4.0]",11,"['make a choice and proceed with recipe', 'dep...",autumn is my favorite time of year to cook! th...,"['winter squash', 'mexican seasoning', 'mixed ...",7,51.5,0.0,13.0,0.0,2.0,0.0,4.0
1,a bit different breakfast pizza,31490,30,26278,2002-06-17,"['30-minutes-or-less', 'time-to-make', 'course...","[173.4, 18.0, 0.0, 17.0, 22.0, 35.0, 1.0]",9,"['preheat oven to 425 degrees f', 'press dough...",this recipe calls for the crust to be prebaked...,"['prepared pizza crust', 'sausage patty', 'egg...",6,173.4,18.0,0.0,17.0,22.0,35.0,1.0
2,all in the kitchen chili,112140,130,196586,2005-02-25,"['time-to-make', 'course', 'preparation', 'mai...","[269.8, 22.0, 32.0, 48.0, 39.0, 27.0, 5.0]",6,"['brown ground beef in large pot', 'add choppe...",this modified version of 'mom's' chili was a h...,"['ground beef', 'yellow onions', 'diced tomato...",13,269.8,22.0,32.0,48.0,39.0,27.0,5.0
3,alouette potatoes,59389,45,68585,2003-04-14,"['60-minutes-or-less', 'time-to-make', 'course...","[368.1, 17.0, 10.0, 2.0, 14.0, 8.0, 20.0]",11,['place potatoes in a large pot of lightly sal...,"this is a super easy, great tasting, make ahea...","['spreadable cheese with garlic and herbs', 'n...",11,368.1,17.0,10.0,2.0,14.0,8.0,20.0
4,amish tomato ketchup for canning,44061,190,41706,2002-10-25,"['weeknight', 'time-to-make', 'course', 'main-...","[352.9, 1.0, 337.0, 23.0, 3.0, 0.0, 28.0]",5,['mix all ingredients& boil for 2 1 / 2 hours ...,my dh's amish mother raised him on this recipe...,"['tomato juice', 'apple cider vinegar', 'sugar...",8,352.9,1.0,337.0,23.0,3.0,0.0,28.0


In [19]:
#convert nutritional info to floats
nutrition_cols = ["calories", "total_fat_pdv", "sugar_pdv", "sodium_pdv", "protein_pdv", "saturated_fat_pdv", "carbs_pdv"]
recipes_df[nutrition_cols] = recipes_df[nutrition_cols].astype(float)

In [20]:
# Turn ingredients into a list of strings

#remove the brackets 
recipes_df["ingredients"] = recipes_df["ingredients"].str.strip("[")
recipes_df["ingredients"] = recipes_df["ingredients"].str.strip("]")
recipes_df["ingredients"][0][0]

"'"

In [21]:
recipes_df["ingredients"] = recipes_df["ingredients"].str.replace("'", "")
recipes_df.head()

,name,recipe_id,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients,calories,total_fat_pdv,sugar_pdv,sodium_pdv,protein_pdv,saturated_fat_pdv,carbs_pdv
0,arriba baked winter squash mexican style,137739,55,47892,2005-09-16,"['60-minutes-or-less', 'time-to-make', 'course...","[51.5, 0.0, 13.0, 0.0, 2.0, 0.0, 4.0]",11,"['make a choice and proceed with recipe', 'dep...",autumn is my favorite time of year to cook! th...,"winter squash, mexican seasoning, mixed spice,...",7,51.5,0.0,13.0,0.0,2.0,0.0,4.0
1,a bit different breakfast pizza,31490,30,26278,2002-06-17,"['30-minutes-or-less', 'time-to-make', 'course...","[173.4, 18.0, 0.0, 17.0, 22.0, 35.0, 1.0]",9,"['preheat oven to 425 degrees f', 'press dough...",this recipe calls for the crust to be prebaked...,"prepared pizza crust, sausage patty, eggs, mil...",6,173.4,18.0,0.0,17.0,22.0,35.0,1.0
2,all in the kitchen chili,112140,130,196586,2005-02-25,"['time-to-make', 'course', 'preparation', 'mai...","[269.8, 22.0, 32.0, 48.0, 39.0, 27.0, 5.0]",6,"['brown ground beef in large pot', 'add choppe...",this modified version of 'mom's' chili was a h...,"ground beef, yellow onions, diced tomatoes, to...",13,269.8,22.0,32.0,48.0,39.0,27.0,5.0
3,alouette potatoes,59389,45,68585,2003-04-14,"['60-minutes-or-less', 'time-to-make', 'course...","[368.1, 17.0, 10.0, 2.0, 14.0, 8.0, 20.0]",11,['place potatoes in a large pot of lightly sal...,"this is a super easy, great tasting, make ahea...","spreadable cheese with garlic and herbs, new p...",11,368.1,17.0,10.0,2.0,14.0,8.0,20.0
4,amish tomato ketchup for canning,44061,190,41706,2002-10-25,"['weeknight', 'time-to-make', 'course', 'main-...","[352.9, 1.0, 337.0, 23.0, 3.0, 0.0, 28.0]",5,['mix all ingredients& boil for 2 1 / 2 hours ...,my dh's amish mother raised him on this recipe...,"tomato juice, apple cider vinegar, sugar, salt...",8,352.9,1.0,337.0,23.0,3.0,0.0,28.0


In [22]:
recipes_df["ingredients"] = recipes_df["ingredients"].str.split(", ")

In [23]:
recipes_df["ingredients"][0][1]

'mexican seasoning'

In [10]:
recipes_df.head()

,name,recipe_id,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients,calories,total_fat_pdv,sugar_pdv,sodium_pdv,protein_pdv,saturated_fat_pdv,carbs_pdv
0,arriba baked winter squash mexican style,137739,55,47892,2005-09-16,"['60-minutes-or-less', 'time-to-make', 'course...","[51.5, 0.0, 13.0, 0.0, 2.0, 0.0, 4.0]",11,"['make a choice and proceed with recipe', 'dep...",autumn is my favorite time of year to cook! th...,"['winter squash', 'mexican seasoning', 'mixed ...",7,51.5,0.0,13.0,0.0,2.0,0.0,4.0
1,a bit different breakfast pizza,31490,30,26278,2002-06-17,"['30-minutes-or-less', 'time-to-make', 'course...","[173.4, 18.0, 0.0, 17.0, 22.0, 35.0, 1.0]",9,"['preheat oven to 425 degrees f', 'press dough...",this recipe calls for the crust to be prebaked...,"['prepared pizza crust', 'sausage patty', 'egg...",6,173.4,18.0,0.0,17.0,22.0,35.0,1.0
2,all in the kitchen chili,112140,130,196586,2005-02-25,"['time-to-make', 'course', 'preparation', 'mai...","[269.8, 22.0, 32.0, 48.0, 39.0, 27.0, 5.0]",6,"['brown ground beef in large pot', 'add choppe...",this modified version of 'mom's' chili was a h...,"['ground beef', 'yellow onions', 'diced tomato...",13,269.8,22.0,32.0,48.0,39.0,27.0,5.0
3,alouette potatoes,59389,45,68585,2003-04-14,"['60-minutes-or-less', 'time-to-make', 'course...","[368.1, 17.0, 10.0, 2.0, 14.0, 8.0, 20.0]",11,['place potatoes in a large pot of lightly sal...,"this is a super easy, great tasting, make ahea...","['spreadable cheese with garlic and herbs', 'n...",11,368.1,17.0,10.0,2.0,14.0,8.0,20.0
4,amish tomato ketchup for canning,44061,190,41706,2002-10-25,"['weeknight', 'time-to-make', 'course', 'main-...","[352.9, 1.0, 337.0, 23.0, 3.0, 0.0, 28.0]",5,['mix all ingredients& boil for 2 1 / 2 hours ...,my dh's amish mother raised him on this recipe...,"['tomato juice', 'apple cider vinegar', 'sugar...",8,352.9,1.0,337.0,23.0,3.0,0.0,28.0


In [ ]:
for row in meta_data_df["categories"]:
    value = row[0][1]
    if value not in subcategories:
        subcategories.append(value) 
subcategories
row["recipe_type"] = recipe_cat

In [28]:
recipes_df.iloc[0]["ingredients"]

['winter squash',
 'mexican seasoning',
 'mixed spice',
 'honey',
 'butter',
 'olive oil',
 'salt']

In [29]:
meat_ingredients = ["chicken", "meat", "turkey", "sausage", "beef", "pork", "ribs" , "pepperoni", "bacon", "ham"]

In [33]:
fake_recipe = ['winter squash',
 'mexican seasoning',
 'mixed spice',
 'honey',
 'butter',
 'olive oil',
 'salt',
"bacon"]

In [37]:
recipe_cat = ["vegetarian"]
meat_tracker = False
for ingredient in meat_ingredients:
    
    for recipe_ingredient in fake_recipe:
        print("checking " + ingredient + " versus " + recipe_ingredient)
        
        if ingredient in recipe_ingredient:
            meat_tracker = True 
            recipe_cat.pop(0)
            
        if meat_tracker:
            break
            
    if meat_tracker:
        break

checking chicken versus winter squash
checking chicken versus mexican seasoning
checking chicken versus mixed spice
checking chicken versus honey
checking chicken versus butter
checking chicken versus olive oil
checking chicken versus salt
checking chicken versus bacon
checking meat versus winter squash
checking meat versus mexican seasoning
checking meat versus mixed spice
checking meat versus honey
checking meat versus butter
checking meat versus olive oil
checking meat versus salt
checking meat versus bacon
checking turkey versus winter squash
checking turkey versus mexican seasoning
checking turkey versus mixed spice
checking turkey versus honey
checking turkey versus butter
checking turkey versus olive oil
checking turkey versus salt
checking turkey versus bacon
checking sausage versus winter squash
checking sausage versus mexican seasoning
checking sausage versus mixed spice
checking sausage versus honey
checking sausage versus butter
checking sausage versus olive oil
checking sa

In [38]:
recipe_cat

[]

In [175]:
recipe_cat = []
dessert_tracker = False
for ingredient in dessert_ingredients:
    for recipe_ingredient in fake_recipe:
        
        print("checking " + ingredient + " versus " + recipe_ingredient)
        if ingredient in recipe_ingredient:
            dessert_tracker = True 
            recipe_cat.append("dessert")
        if dessert_tracker:
            break
    if dessert_tracker:
        break
    
        
            

checking ice cream versus milk
checking ice cream versus frozen apple juice
checking cake versus milk
checking cake versus frozen apple juice
checking cookies versus milk
checking cookies versus frozen apple juice
checking chocolate versus milk
checking chocolate versus frozen apple juice


In [72]:
meat_ingredients = ["chicken", "meat", "turkey", "sausage", "beef", "pork", "ribs" , "pepperoni", "bacon", "ham", 
                       "bone broth", "salami", "lamb", "venison", "duck", "rabbit", "steak", "bison"]
fish_ingredients = ["fish", "salmon", "tilapia", "sole", "scallops", "shrimp", "clams", "mussels", "crawfish",
                       "lobster", "shell fish", "tuna", "cod", "trout"]
non_veg_ingredients = meat_ingredients + fish_ingredients
non_veg_ingredients

['chicken',
 'meat',
 'turkey',
 'sausage',
 'beef',
 'pork',
 'ribs',
 'pepperoni',
 'bacon',
 'ham',
 'bone broth',
 'salami',
 'lamb',
 'venison',
 'duck',
 'rabbit',
 'steak',
 'bison',
 'fish',
 'salmon',
 'tilapia',
 'sole',
 'scallops',
 'shrimp',
 'clams',
 'mussels',
 'crawfish',
 'lobster',
 'shell fish',
 'tuna',
 'cod',
 'trout']

In [176]:
recipe_cat

[]

In [80]:
def categorize(row):
    recipe_cat = ["vegetarian"]
    
    dessert_tracker = False
    veg_tracker = True
    drink_tracker = False

    
    dessert_ingredients = ["ice cream", "cake", "cookies", "chocolate", "wafer"]
    meat_ingredients = ["chicken", "meat", "turkey", "sausage", "beef", "pork", "ribs" , "pepperoni", "bacon", "ham", 
                       "bone broth", "salami", "lamb", "venison", "duck", "rabbit", "steak", "bison"]
    fish_ingredients = ["fish", "salmon", "tilapia", "sole", "scallops", "shrimp", "clams", "mussels", "crawfish",
                       "lobster", "shell fish", "tuna", "cod", "trout"]
    non_veg_ingredients = meat_ingredients + fish_ingredients
    beverage_types = ["smoothie", "drink", "milkshake", "cocktail", "mocktail", "spritz", "juice"]
    
    if row["calories"] <= 400:
        recipe_cat.append("low cal")
    if row["carbs_pdv"] <= 10:
        recipe_cat.append("low carb")
    if row["sugar_pdv"] == 0:
        recipe_cat.append("sugar free")
    
    for i in beverage_types:
        if i in row["name"]:
            drink_tracker = True
            recipe_cat.append("beverage")
        if drink_tracker:
            break
    
    #for i in beverage_names:
        #for recipe_description in row["name"]:
            #if i in beverage_names:
                #drink_tracker = True 
                #recipe_cat.append("beverage")
            #if drink_tracker:
                #break
        #if drink_tracker:
            #break
        
    for ingredient in dessert_ingredients:
        for recipe_ingredient in row["ingredients"]:
            if ingredient in recipe_ingredient:
                dessert_tracker = True 
                recipe_cat.pop(0)
                recipe_cat.append("dessert")
            if dessert_tracker:
                break
        if dessert_tracker:
            break
    
    for ingredient in non_veg_ingredients:
        for recipe_ingredient in row["ingredients"]: 
            if ingredient in recipe_ingredient:
                veg_tracker = False 
                recipe_cat.pop(0)
                recipe_cat.append("not vegetarian")
            if veg_tracker == False:
                break   
        if veg_tracker == False:
            break
    
    return recipe_cat


In [81]:
# Create new column and apply the categorize function to fill with subcategories
recipes_df["recipe_type"] = recipes_df.apply(lambda row: categorize(row), axis=1)

TypeError: 'float' object is not iterable

In [75]:
recipes_df["recipe_type"].head(10)

0                    [vegetarian, low cal, low carb]
1    [low cal, low carb, sugar free, not vegetarian]
2                [low cal, low carb, not vegetarian]
3                              [vegetarian, low cal]
4                              [vegetarian, low cal]
5                       [low cal, low carb, dessert]
6                    [vegetarian, low cal, low carb]
7                                   [not vegetarian]
8                                          [dessert]
9                                          [dessert]
Name: recipe_type, dtype: object

In [70]:
recipes_df.iloc[9]

['sugar',
 'unsalted butter',
 'bananas',
 'eggs',
 'fresh lemon juice',
 'orange rind',
 'cake flour',
 'baking soda',
 'salt']

In [58]:
recipes_df.drop(["contributor_id", "nutrition", "n_steps", "n_ingredients", "submitted"], axis=1, inplace=True)

In [60]:
recipes_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 231637 entries, 0 to 231636
Data columns (total 15 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   name               231636 non-null  object 
 1   recipe_id          231637 non-null  object 
 2   minutes            231637 non-null  int64  
 3   tags               231637 non-null  object 
 4   steps              231637 non-null  object 
 5   description        226658 non-null  object 
 6   ingredients        231637 non-null  object 
 7   calories           231637 non-null  float64
 8   total_fat_pdv      231637 non-null  float64
 9   sugar_pdv          231637 non-null  float64
 10  sodium_pdv         231637 non-null  float64
 11  protein_pdv        231637 non-null  float64
 12  saturated_fat_pdv  231637 non-null  float64
 13  carbs_pdv          231637 non-null  float64
 14  recipe_type        231637 non-null  object 
dtypes: float64(7), int64(1), object(7)
memory usage: 26